In [1]:
import os
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision

from data import loader_train, loader_val, loader_test
from train import train_model
from models import RadiographyClassifier

In [5]:
from data import contrast_transform
# Plot an image
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    img = contrast_transform(img)
    plt.imshow(np.transpose(img.numpy(), (1, 2, 0)))
    plt.show()
    

In [ ]:
# Iteración de prueba para ver imágenes y sus clasificaciones
train_loader = loader_train



model = RadiographyClassifier(5)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Imprimir la etiqueta real y la etiqueta predicha
model.eval()  # Establecer el modelo en modo de evaluación

for inputs, labels in train_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    mod_input = torch.stack([contrast_transform(x) for x in inputs])

    outputs = model(inputs, mod_input)
    print(outputs[0])
    imshow(torchvision.utils.make_grid(inputs[0]))
    print(labels[0])

    break


In [ ]:
#train a model
model = RadiographyClassifier(5)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_model(model, loader_train, loader_val, criterion, optimizer, device, num_epochs=10)